In [1]:
from sklearn import model_selection, preprocessing, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import csv
import pandas as pd, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import nltk
#nltk.download()

def _devdataset(file):
    train = csv.reader(open(file))
    texts, labels1, labelscc = [], [], []
    
    for row in train:
        texts.append(row[1])
        labels1.append(row[2])
        labelscc.append(int(row[3][1:]))
            
    return texts, labels1, labelscc
    

trainDF = pd.DataFrame()
trainDF['text'], trainDF['label'], trainDF['labelcc'] = _devdataset('EIN2.csv')

Using TensorFlow backend.


FileNotFoundError: [Errno 2] No such file or directory: 'EIN2.csv'

In [ ]:
import statistics

data=pd.read_csv("/Users/Rushi/Documents/GRAFall2018/%Dist/DistributionMajorGroup.csv")
samplesize = pd.DataFrame()

for yr in range(1989,2016):
    count = pd.to_numeric(data[str(yr)].drop(labels=[0,27])).apply(lambda x: int(x/2))
    samplesize.insert(loc=yr-1989, column=yr, value=count)
'''    
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y), metrics.precision_score(predictions, valid_y, average='weighted'), metrics.recall_score(predictions, valid_y, average='weighted')

'''
def dataformodel(trainDF):
    # split the dataset into training and validation datasets 
    train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

    # create a count vectorizer object 
    count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
    count_vect.fit(trainDF['text'])

    # transform the training and validation data using count vectorizer object
    xtrain_count =  count_vect.transform(train_x)
    xvalid_count =  count_vect.transform(valid_x)  
    
    # word level tf-idf
    tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
    tfidf_vect.fit(trainDF['text'])
    xtrain_tfidf =  tfidf_vect.transform(train_x)
    xvalid_tfidf =  tfidf_vect.transform(valid_x)
    
    # ngram level tf-idf 
    tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
    tfidf_vect_ngram.fit(trainDF['text'])
    xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
    xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)
    
    
    def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
        # fit the training dataset on the classifier
        classifier.fit(feature_vector_train, label)
    
        # predict the labels on validation dataset
        predictions = classifier.predict(feature_vector_valid)
    
        if is_neural_net:
            predictions = predictions.argmax(axis=-1)
        
        return metrics.accuracy_score(predictions, valid_y), metrics.precision_score(predictions, valid_y, average='weighted'), metrics.recall_score(predictions, valid_y, average='weighted')

    def create_model_architecture(input_size):
        # create input layer 
        input_layer = layers.Input((input_size, ), sparse=True)

        # create hidden layer
        hidden_layer = layers.Dense(100, activation="relu")(input_layer)

        # create output layer
        output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

        classifier = models.Model(inputs = input_layer, outputs = output_layer)
        classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
        return classifier

    # Naive Bayes on Count Vectors
    acNB, prNB, rcNB = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
    #print("NB, Count Vectors: ", accuracyNB)

    # RF on Count Vectors
    acRF, prRF, rcRF = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
    #print("RF, Count Vectors: ", accuracyRF)
    
    # label encode the target variable 
    encoder = preprocessing.LabelEncoder()
    train_y = encoder.fit_transform(train_y)
    valid_y = encoder.fit_transform(valid_y)
        
    classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
    acNN, prNN, rcNN = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
    #print("NN, Ngram Level TF IDF Vectors",  accuracyNN)
    
    return [acNB, acRF, acNN, prNB, prRF, prNN, rcNB, rcRF, rcNN]
    

year = 2015
for i in range(1,27):
    print('class: ', i) 
    #7
    size=int(samplesize[year][i])
    train=trainDF.loc[trainDF['label']==chr(i+64)]
    accuracy, precision, recall = [[],[],[]], [[],[],[]], [[],[],[]]
    #8
    for count in range(1, 1001):
        train = train.sample(frac=1)
        result=dataformodel(train)
        for j in range(0,3):
            accuracy[j].append(result[j])
            precision[j].append(result[j+3])
            recall[j].append(result[j+6])
    stats = [[], [], []]                                                                                                                                        
    for j in range(0,3):
        stats[0].append([statistics.mean(accuracy[j]), statistics.stdev(accuracy[j])])                                                                                                                                     
        stats[1].append([statistcis.mean(precision[j]), statistics.stdev(precision[j])])                                                                                                                                             
        stats[2].append([statistics.mane(recall[j]), statistics.stdev(recall[j])])                                                                                                                                         
    
    stats = pd.DataFrame(stats)                                                                                                                                             
    print(stats)
    with open('statsbyyear.csv', 'a') as f:
          stats.to_csv(f, header=False)                
        
    result = pd.DataFrame(result, columns=['acNB','acRF', 'acNN', 'prNB', 'prRF', 'prNN', 'rcNB', 'rcRF', 'rcNN'])
    result.to_csv(results.csv)